In [1]:
import requests
import pandas as pd
from itertools import product

# URL til SSB API for tabell 03024
api_url = "https://data.ssb.no/api/v0/no/table/03024/"

# Definerer API-spørringen
query = {
    "query": [],
    "response": {
        "format": "json-stat2"
    }
}

# Hente data fra API
response = requests.post(api_url, json=query)

# Sjekke om forespørselen var vellykket
if response.status_code == 200:
    data = response.json()
    
    # Ekstrahere dimensjonsnavn og verdier
    dimension_names = list(data["dimension"].keys())
    values = data["value"]
    
    # Opprette en liste av kombinasjoner av dimensjoner
    dimensions = [list(data["dimension"][dim]["category"]["label"].values()) for dim in dimension_names]
    
    # Generere alle kombinasjoner av dimensjonsverdier
    all_combinations = list(product(*dimensions))
    
    # Lage en DataFrame med riktige dimensjoner
    df = pd.DataFrame(all_combinations, columns=dimension_names)
    df["Value"] = values  # Knytte verdiene til riktige rader
    
    # Pivotere data for å få ønsket struktur
    df_pivot = df.pivot_table(index="Tid", columns=["VareGrupper2", "ContentsCode"], values="Value", aggfunc="sum").reset_index()
    
    # Gi kolonnene mer forståelige navn med riktig rekkefølge
    df_pivot.columns = ["Tid", "Fersk laks - Kilospris", "Fersk laks - Vekt (tonn)", "Frosset laks - Kilospris", "Frosset laks - Vekt (tonn)"]
    


    
    # Vise tabellen i et lesbart format
    print(df_pivot.to_string(index=False))

    
    print("Data hentet, sortert og formatert for visning.")
else:
    print(f"Feil ved henting av data: {response.status_code}")
    print(response.text)  # Skriv ut feilmeldingen fra API-et




    Tid  Fersk laks - Kilospris  Fersk laks - Vekt (tonn)  Frosset laks - Kilospris  Frosset laks - Vekt (tonn)
2000U01                   30.98                    3728.0                     32.54                       383.0
2000U02                   31.12                    4054.0                     33.63                       216.0
2000U03                   31.03                    4043.0                     36.06                       633.0
2000U04                   30.95                    3730.0                     34.27                       393.0
2000U05                   31.30                    3831.0                     33.91                       453.0
2000U06                   32.53                    4415.0                     35.30                       265.0
2000U07                   32.46                    4617.0                     34.49                       344.0
2000U08                   32.19                    4463.0                     36.34                     

In [2]:
#manipulerer dataene ved hjelp av lambda funksjoner 
#legger til to kolonner ekstra 

df_pivot["Kilopris_fersk_laks_2"] = df_pivot.apply(lambda x: x["Fersk laks - Kilospris"] * 100, axis=1)
df_pivot["Vekt_fersk_laks_2"] = df_pivot.apply(lambda x: x["Fersk laks - Vekt (tonn)"] * 50, axis=1)
df_pivot.head(1000)

,Tid,Fersk laks - Kilospris,Fersk laks - Vekt (tonn),Frosset laks - Kilospris,Frosset laks - Vekt (tonn),Kilopris_fersk_laks_2,Vekt_fersk_laks_2
0,2000U01,30.98,3728.0,32.54,383.0,3098.0,186400.0
1,2000U02,31.12,4054.0,33.63,216.0,3112.0,202700.0
2,2000U03,31.03,4043.0,36.06,633.0,3103.0,202150.0
3,2000U04,30.95,3730.0,34.27,393.0,3095.0,186500.0
4,2000U05,31.30,3831.0,33.91,453.0,3130.0,191550.0
...,...,...,...,...,...,...,...
995,2019U05,57.41,16333.0,56.43,315.0,5741.0,816650.0
996,2019U06,56.73,17688.0,63.17,250.0,5673.0,884400.0
997,2019U07,56.82,17634.0,62.63,284.0,5682.0,881700.0
998,2019U08,60.00,17103.0,63.51,315.0,6000.0,855150.0


In [3]:
#regner et gjennomsnitt av kiloprisen for fersk laks
print(df_pivot["Fersk laks - Kilospris"].mean())

#regner øverste og laveste pris for fersk laks
print(df_pivot["Fersk laks - Kilospris"].max())
print(df_pivot["Fersk laks - Kilospris"].min())


#lager en funksjon med en if-setning som sier om laksen er dyr eller billig 
df_pivot["Kilopris fersk laks"] = df_pivot["Fersk laks - Kilospris"].apply(lambda x: "Dyr" if x > 90 else "normal" if x > 45 else "Billig")
print(df_pivot.to_string(index=False))


45.35902050113895
125.36
17.46
    Tid  Fersk laks - Kilospris  Fersk laks - Vekt (tonn)  Frosset laks - Kilospris  Frosset laks - Vekt (tonn)  Kilopris_fersk_laks_2  Vekt_fersk_laks_2 Kilopris fersk laks
2000U01                   30.98                    3728.0                     32.54                       383.0                 3098.0           186400.0              Billig
2000U02                   31.12                    4054.0                     33.63                       216.0                 3112.0           202700.0              Billig
2000U03                   31.03                    4043.0                     36.06                       633.0                 3103.0           202150.0              Billig
2000U04                   30.95                    3730.0                     34.27                       393.0                 3095.0           186500.0              Billig
2000U05                   31.30                    3831.0                     33.91                